# ANOVA docomposition of a general function

Functional ANOVA decomposition represents a high-dimensional function as a function of the form
$$f(x_1, x_2, \ldots, x_D) = f_0 + \sum_{i=1}^D f_i(x_i) + \sum_{i<j} f_{ij}(x_i, x_j) + \sum_{i<j<k} f_{ijk}(x_i, x_j, x_k) + \cdots + f_{1,\ldots,D}(x_1, \ldots, x_D).$$


Let's first talk about how to compute the ANOVA components in general and then we focus on how to do it for $f(x_1, \ldots, x_D)$ which is evaluated as a mean of a Gaussian process.

The first step is to choose the projection operator $P$:
$$Pf := \int_{[a,b]}f(x)d\mu(x)$$
We are going to use the projection operator $P$ using Lebegue measure $Pf := \int_{[a,b]} f(x) dx$, so all intergrals should work out as expected.

## Two-dimensional case

Now we use the projection operator to define the constant and the main effects. We assume $D=2$ for now and then introduce some more notation to generalize:
\begin{align}
f_0 & = \int_{[a_1, b_1]} \int_{[a_2,b_2]} f(x_1, x_2) dx_1 dx_2 \\
f_1 (x_1) & = \int_{[a_2, b_2]} f(x_1, x_2) dx_2 - f_0 \\
f_2 (x_2) & = \int_{[a_1, b_1]} f(x_1, x_2) dx_1 - f_0 
\end{align}

The interaction effect $f_{1,2}(x_1,x_2)$ is defined as the remainder to make the ANOVA decomposition to work out correctly:
$$f_{1,2}(x_1,x_2) = f(x_1, x_2) - f_0 - f_1(x_1) - f_2(x_2).$$

The **total variance** (TV) of the predictor is defined as
$$\sigma^2(f) := \int_{[a_1, b_1]} \int_{[a_2,b_2]} (f(x_1, x_2) - f_0)^2 d x_1 d x_2$$

One can show that TV is decomposible into the sum of variances of main effects and interactions defined above:
\begin{align}
\sigma_1^2(f_1) &:= \int_{[a_1, b_1]} (f_1(x_1) - f_0)^2 d x_1 \\
\sigma_2^2(f_2) &:= \int_{[a_2, b_2]} (f_2(x_2) - f_0)^2 d x_2 \\
\sigma_{1,2}^2(f_{1,2}) &:= \int_{[a_1, b_1]} \int_{[a_2,b_2]} \left(f_{1,2}(x_1, x_2) - f_0 - \left[f_1(x_1)-f_0\right] - \left[f_2(x_2)-f_0\right]\right)^2 dx_1 dx_2 \\ 
\sigma^2(f) &= \sigma_1^2(f_1) + \sigma_2^2(f_2) + \sigma_{1,2}^2 (f_{1,2}).
\end{align}

And so, by dividing individual variances by TV we can express these compoents as percentages.

## General case

Using subsets $\vec{u}\subseteq \{1,\ldots, D\}$, we can establish a shorthand notation for ANOVA components, where $f_\vec{u}$ and $\vec{x}_\vec{u}$ represents a subset of vector $\vec{x}$ with components $x_i, i \in \vec{u}$. The we have
\begin{align}
f(x_1, \ldots, x_D) &= \sum_{\vec\subseteq\{1\ldots,D\}} f_{\vec{u}}(x_{\vec{u}}),\\
f_{\vec{u}}(x_{\vec{u}}) &= \Bigl(\prod_{j\in\{1,\ldots,D\}\setminus \vec{u}} P_j f\Bigr) - \sum_{\vec{v}\subset \vec{u}}f_{\vec{v}}(x_{\vec{v}}) = \sum_{\vec{v}\subseteq\vec{u}}(-1)^{|u| - |v|} \Bigl(\prod_{j\in\{1,\ldots,D\}\setminus \vec{v}} P_j f\Bigr)(x_{\vec{v}})\\
\sigma^2(f_\vec{u}) &= \int_{\Omega^D} \Bigl(f_\vec{u}(x_\vec{u}))^2dx \\
\sigma^2(f) &= \int_{\Omega^D} \Bigl(f(x) - f_0\Bigr)^2 dx = \sum_{\vec{u} \subseteq \{1,\ldots,D\}, \vec{u}\neq\emptyset}\sigma^2(f_\vec{u}).
\end{align}

## ANOVA for Gaussian process

For Bayesian optimization we are using the Gaussian proces defined by a parametrized RBF kernel of the form
$$
K(x,x'|\theta_0, \vec{\theta}_1) = \theta_0^2 \exp\Bigl(-(x-x')^t D(\vec{\theta}_1) (x-x'))\Bigr),
$$
where $[D(\vec{\theta}_1)]_{ij} = (\theta_{1,i}^2 + \epsilon)\delta_{ij}$ is a diagonal matrix of scaling coefficients, $\epsilon$ beeing a constant and $\delta_{ij}$ beeing Dirac-delta.
$$
K(x,x'|\theta_0, \vec{\theta}_1) = \theta_0^2\exp\Bigl(-\sum_{i=1}^D (\theta_{1i}^2+\epsilon)(x_i-x'_i)^2\Bigr). 
$$

I drop $\theta$'s from the definition of $K$ for shorthand.

Let $\Sigma$ be the correlation matrix of the training set $\{(x_i, y_i)\}_{i=1}^n$ and $k(x)$ a vector with correlations to the new point $x$, to evaluate the functional given by the GP and get the mean at the points $x$  we do:
\begin{align}
\Sigma &:= \{K(x_i, x_j)\}_{ij} \\
k(x) &:= \{K(x,x_i)\}_{i=1}^n \\
f(x) &:= k(x)^t \Sigma^{-1}y \\
&= \sum_{i=1}^n K(x, x_i) \Sigma_{i}^{-1}y,
\end{align}
where $\Sigma_i^{-1}$ is the $i$'th row of the inverse correlation matrix (of course, we are just solving $\Sigma z = y$ and take the $i$'th component of $z$, so I will just use $z_i$ as a shorthand.